<H1>Create a simple line beam model</H1>
Run the whole code to generate and solve the model and display the minimum sagging moment, 3 different ways

In [25]:
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.0")

In [26]:
# Input Params
length = 10.0 #m		Length of beam
loading = 10.0  #kN/m   Uniformly distibuted load
model_name = "myBeamModel" 

When interacting with Modeller externally a large speed up can be achived by setting the model to be invisible. Take care to restore the visibility and watch out for exceptions in the code that can leave Modeller running in the background.

In [27]:
if 0: # To speed up interaction with Modeller from python set the app invisible (toggle this to true)
    lusas.setVisible(False)

In [28]:
# Create a new model
db = lusas.newDatabase("Structural", f"{model_name}.mdl")
# Set the vertical axis
db.setLogicalUpAxis("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

In [29]:
''' *** Create a line ***'''
# Get the geometry data object
geomData = lusas.geometryData()
# Set the defaults
geomData.setAllDefaults()
# Set the coordinates of the first point
geomData.addCoords(0, 0, 0)
# Set the coordinates of the second point
geomData.addCoords(length, 0, 0)
# Create the line object from the geomety data
linesDBop = win32.CastTo(db.createLine(geomData), "IFObjectSet") # Modeller returns the baseclass IFDatabaseOperations, we need to upcast
# Get the lines
lines = linesDBop.getObjects("Lines", "All")
# Get a reference to the created line, there is only one here but there can be many depending on the coords in geomdata
beamLine = lines[0]

In [ ]:
''' *** Create a mesh attribute ***'''
meshAttr = db.createMeshLine("Beam Mesh")
# Set the element type and number of elements (3 thick beam elements here)
meshAttr.setNumber("BMI21", 3)

In [ ]:
''' *** Create a geometric attribute ***'''
geomAttr = db.createGeometricLine("Beam Geometry")
# Set the element type
geomAttr.setValue("elementType", "3D Thick Beam")
# Set the beam section properties
geomAttr.setBeam(0.0125, 0.0004573, 0.00002347, 0.0, 0.00000121, 0.00532608, 0.00755776, 0.0, 0.0, 0)

In [32]:
''' *** Create a material attribute ***'''
materialAttr = db.createIsotropicMaterial("Steel", 209000000.0, 0.3, 7.8)

In [ ]:
''' *** Create a support attribute (fixed) *** '''
fixedSupport = db.createSupportStructural("Fixed")
# set the freedoms F=Free, R=Restrained
fixedSupport.setStructural("R", "R", "R", "F", "F", "F", "F", "F", "F")

''' *** Create a support attribute (pinned) *** '''
pinnedSupport = db.createSupportStructural("Pinned")
# set the freedoms 
pinnedSupport.setStructural("F", "R", "R", "F", "F", "F", "F", "F", "F")

In [ ]:
''' *** Create a beam load attribute *** '''
loadAttr = db.createLoadingBeamDistributed("UDL")
loadAttr.setBeamDistributed("Parametric", "Global", "beam")
loadAttr.addRow(0.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0)

In [ ]:
''' *** Assign the attributes to the geometry *** '''
# get the assignment object
assignment = lusas.assignment()
# set the defaults
assignment.setAllDefaults()

# Assign the mesh
meshAttr.assignTo(beamLine, assignment)
# Assign the geometry
geomAttr.assignTo(beamLine, assignment)
# Assign the material
materialAttr.assignTo(beamLine, assignment)
# Assign the loading
loadAttr.assignTo(beamLine, assignment)

''' Assign the supports to the points of the line '''
# Assign the fixed support to the first point
fixedSupport.assignTo(beamLine.getStartPoint(), assignment)
# Assign the pinned support to the last point
pinnedSupport.assignTo(beamLine.getEndPoint(), assignment)

<H2>Solving the Analysis</H2>

In [36]:
import sys
''' The model is ready to be solved '''
db.save()


for analysis in db.getAnalyses():
    # File paths for the data file and results file
    datPath = analysis.getFilenameNoExtension() + ".dat"
    mysPath = analysis.getFilenameNoExtension() + ".mys"

    # Get the solverOptions object and set the defaults
    solverOptions = lusas.solverOptions()
    solverOptions.setAllDefaults()

    # Get the exporter object that will export the model to solver
    solverExport = lusas.solverExport()
    # Set the analysis name
    solverExport.setAllDefaults()
    solverExport.setAnalysis(analysis.getName())

    # Export the model as a solver data file (.dat)
    returnCode = db.exportSolver(solverExport, solverOptions)
    if (returnCode != 0) :
        # If the export fails we cannot run the analysis
        print(f"Tabulation of the data file failed with return code {returnCode}")
        sys.exit(returnCode)

    # If we sucessfully exported the .dat file we can run the analysis
    returnCode = lusas.solve(datPath, solverOptions)
    if (returnCode != 0) :
        # If the analysis fails we cannot access the results
        print(f"The analysis failed with return code {returnCode}")
        sys.exit(returnCode)
    else:
        # if the analysis is sucessful load the results
        db.openResults(mysPath)
        db.getResultsCache().calculateNow()

In [37]:
''' *** Successful analysis - Process the results to determine the max bending *** '''
min_moment = sys.float_info.max
nodeNum = 0
# Get the results at each node to determine the max
for element in beamLine.getElements():
    for node in element.getNodes():
        # Extract the nodal result for the required Entity and Component
        my = node.getResults("Force/Moment - Thick 3D Beam", "My")[0] # returns a tuple, take the actual value
        # Save the minimum (sagging) moment -ve in LUSAS
        if my < min_moment :
            min_moment = my
            nodeNum = node.getID()

In [ ]:
# Get the units of the current model for display
forceUnit = db.getModelUnits().getForceShortName()
lengthUnit = db.getModelUnits().getLengthShortName()

# Print the result
print("Max Sagging Bending Moment (Nodal) = {0:.1f}{1}{2} at node {3}".format(min_moment, forceUnit, lengthUnit, nodeNum) )

In [ ]:
# BMI21 Elements have results at "internal points" between nodes, get the results here too
min_moment2 = sys.float_info.max
elementNum = 0
internalPnt = 0
for element in beamLine.getElements():
	if element.shouldUseInternalPoints(): # We know they should but good practice to check incase this code is used elsewhere
		for i in range(0, element.countInternalPoints()):
			# Extract the nodal result for the required Entity and Component
			my = element.getInternalResults(i, "Force/Moment - Thick 3D Beam", "My")[0] # returns a tuple, take the actual value
			# Save the minimum (sagging) moment -ve in LUSAS
			if my < min_moment2 :
				min_moment2 = my
				elementNum = element.getID()
				internalPnt = i
				
# Print the result using the lusas string format
print("Max Sagging Bending Moment (Internal Point) = {0:.1f}{1}{2} at element {3}, ip {4}".format(min_moment2, forceUnit, lengthUnit, elementNum, internalPnt) )

Alternatively we can simply ask the line to do this work for us

In [ ]:
min_moment3 = beamLine.getResults("featureMin", "Force/Moment - Thick 3D Beam", "My", None, elementNum, internalPnt)

print("Max Sagging Bending Moment (Feature) = {0:.1f}{1}{2} at element {3}, ip {4}".format(min_moment2, forceUnit, lengthUnit, elementNum, internalPnt) )
